In [ ]:
import geemap
import ee

In [ ]:
Map = geemap.Map()
Map

In [ ]:
jiangxi = ee.FeatureCollection('users/2431566134liumonarch/monarch/Jiangxi')

In [ ]:
empty = ee.Image().byte()

JX_img = empty.paint(
    featureCollection=jiangxi,
    color='ID'
)
palette = ['FF0000', '00FF00', '0000FF']
Map.addLayer(JX_img, {'palette': palette, 'max': 12, 'min': 0}, 'edsg')

In [ ]:
Map.addLayer(jiangxi, {}, 'jx')
Map.center_object(jiangxi, 6)

In [ ]:
jiangxi = jiangxi.sort('ID')

jx_List = jiangxi.toList(4)

In [ ]:
Points = []
for i in range(4):
    feature1 = ee.Feature(jx_List.get(i))
    name = feature1.get('CityNameC')
    ID = feature1.get('ID')
    def setID(fea):
        return fea.set('name', name).set('ID', ID)
    temp = ee.FeatureCollection.randomPoints(
        ee.FeatureCollection.flatten(jx_List).geometry(), 300).map(setID)
    Points.append(temp)

In [ ]:
Points1 = ee.FeatureCollection.flatten(Points)

In [ ]:
Points1.first().getInfo()

In [ ]:
path_csv = 'F:/temp.csv'

# geemap.zonal_statistics(JX_img, Points1, path_csv, scale=10)

In [ ]:
import pandas as pd
data = pd.read_csv(path_csv, sep=',')

In [ ]:
data1 = []
for i in range(11):
    temp = []
    for j in range(11):
        x = len(data[(data['mean']==i) & (data['ID']==j)])
        temp.append(x)
    data1.append(temp)

In [ ]:
df = pd.DataFrame(data1)

index = data['name'].unique()
index = index[::-1]
index

df.index = index

df.columns = index

In [ ]:
df

In [ ]:
df['Total'] = df.sum(axis=1)

df = df.T
df['Total'] = df.sum(axis=1)
df = df.T

In [ ]:
Sd = 0
PAs = []
UAs = []
Fs = []
for i in df.columns[:-1]:
    for j in df.index[:-1]:
        if i == j:
            PA = df[i][j]/df[i]['Total']*100
            UA = df[i][j]/df['Total'][j]*100
            F = (PA*UA)/(UA+PA)*2
            PA = format(PA, '.2f') + '%'
            PAs.append(PA)
            Sd += df[i][j]
            UA = format(UA, '.2f') + '%'
            UAs.append(UA)
            F = format(F, '.2f') + '%'
            Fs.append(F)

In [ ]:
OA = format(Sd / df['Total']['Total'] * 100, '.2f') + '%'

In [ ]:
UAs.append(np.nan)

In [ ]:
df['User Accuracy'] = UAs

In [ ]:
df

In [ ]:
df.append([PAs])